In [468]:
import numpy as np
import glob
import os
import pandas as pd
import datefinder
import cv2
import random

In [469]:
def correct(text,field="total"):
    address_pattern = {"VIA": "VM+",
                    "VIA%": "VM+",
                    "VIA4": "VM+",
                    "VIA?": "VM+",
                    "VIM": "VM+",
                    "VIM%": "VM+",
                    "VIM4": "VM+",
                    "VIM?": "VM+",
                    "VIN%": "VM+",
                    "VM4": "VM+",
                    "VM?": "VM+",
                    "VMẻ": "VM+",
                    "VN%": "VM+",
                    "VM%" :"VM+" ,
                      "VMN" :"VM+",
                      'VMc' : "VM+",
                      'ONỈ'  : 'CHỈ'}
    
    
    
    company_patterns = { 
    "ĐHỊ" : 'ĐẠI',
    }
    correct_patterns = {
        'TIÊN': 'TIỀN',
        'TTỔNG': 'TỔNG',
        'Tiên': 'Tiền',
        'TIẾN': 'TIỀN',
        '11ỀN': 'TIỀN',
        'TSTOÁN': 'T. TOÁN',
        'tiên': 'tiền',
        'toc': 'toán',
        'QUẤY': 'QUẦY',
        'CŨNG': 'CỘNG',
        'Tiến': 'Tiền',
        'OÁN': 'TOÁN',
        'TĂNG': 'TỔNG',
        'TOÁNG': 'TOÁN',
        'TỜNG': 'TỔNG',
        'TÔNG': 'TỔNG',
        'toàn': 'toán',
        'QUÁY': 'QUẦY',
        'T,': 'T.',
        'TOÀN': 'TOÁN',
        'Tpán': 'Toán',
        'CÔNG': 'CỘNG',
        'Ống': 'Tổng',
        'Cộng': 'Cộng',
        'tiến': 'tiền',
        'toan(VND)': 'toan (VND)',
        'stong': 'tong',
        'CŨNG:': 'CỘNG:',
        "TŨNG" :"TỔNG",
        # '-': '',
        '1': '',
        'VÀ': '',
        'tiên:': 'tiền:',
        'QUẤY':'QUẦY',
        '"TỔNG': 'TỔNG',
        'TOÁN"': 'TOÁN',
        'THÀNH' :'THANH',
        'UND': 'VND',
        
    }

    date_patterns = {
        'bản': 'bán',
        'Ngayr': 'Ngay:',
        'họn': 'hẹn',
        'bản:': 'bán:',
        'Bản:': 'Bán:',
        'iểm': 'Điểm',
        'Nat': 'Ngày',
        'Vgày:': 'Ngày',
        'GIÒ:': 'GIỜ:',
        'GIỮ:': 'GIỜ',

    }

    new_line = text.strip()
    if field =="total":
        for word in text.strip().split(' '):
            if word in correct_patterns: # date_patterns for date 
                new_line = new_line.replace(word, correct_patterns.get(word)) # date_patterns for date 
    if field =="date":
        for word in text.strip().split(' '):
            if word in date_patterns: # date_patterns for date 
                new_line = new_line.replace(word, date_patterns.get(word)) # date_patterns for date 
    if field == "address":
        for word in text.strip().split(' '):
            if word in address_pattern: # date_patterns for date 
                new_line = new_line.replace(word, address_pattern.get(word)) 
    if field == "company":
        for word in text.strip().split(' '):
            if word in company_patterns: # date_patterns for date 
                new_line = new_line.replace(word, company_patterns.get(word)) 
        
    return new_line

In [470]:
paths = glob.glob("/media/thorpham/PROJECT/OCR-challenge/preprocessing/graph_model/result_spelling_correction/*")
node_labels = ['other', 'company', 'address', 'date', 'total']
def hasNumbers(inputString):
    inputString = inputString.replace("Đ","")
    return any(char.isdigit() for char in inputString)

def hasCharacters(inputString):
    return any(char.isalpha() for char in inputString)
def check_address(inputString):
    count = 0
    text =  inputString.split(" ")
    for i in text:
        for char in i :
            if char.isdigit() :
                count += 1
    if count > 8 :
        return False
    else :
        return True

In [477]:
data_frame = []
date_str = []
count = 0
for path in paths :
    results = {"company":[],"address":[],"date":[],"total":[]}
    outputs   = {"company":"","address":"","date":"","total":""}
    name = os.path.basename(path)
    name_id = name.replace("txt","jpg")
#     image = cv2.imread(f"/media/thorpham/PROJECT/OCR-challenge/preprocessing/data_for_submit/data_test/{name_id}")
#     image_show = image.copy()
    with open(path,"r") as file :
        data = file.readlines()
        for line in data:
            tmp = line.strip().split("\t")
            x,y = tmp[:2]
            ymax = tmp[5]
            text = tmp[10]
            class_id = tmp[11]
            save = [int(x),int(y),text,class_id,ymax]
            results[class_id].append(save)      
        for key,v in results.items():       
            if key == "date" :
                for j in results[key]:
                    if "Ngày" in j[2] :
                        j[2] =  "Ngày" + j[2].split("Ngày")[1]
                    if "Thời" in j[2] :
                        j[2] =  "Thời" + j[2].split("Thời")[1]
                if len(results[key])==0:
                    with open(f"/media/thorpham/PROJECT/OCR-challenge/preprocessing/data_for_submit/private_test/OCR_spelling_correction/{name}","r") as f :
                        file = f.readlines()
                        for i in file :
                            x1,y1 = i.strip().split("\t")[:2]
                            tm = i.strip().split("\t")[8]
                            tm2 = tm.split(" ")
                            for k in tm2 :
                                if len(k)<10 and (k.count("-") <2 or k.count("/")<2  or k.count(".")<2):
                                    continue
                                else :
                                    try  :
                                        matches = datefinder.find_dates(k)
                                        for match in matches:
#                                             print(tm)           
                                            if "Ngày" in tm:
                                                
                                                results[key].append([int(x1),int(y1),"Ngày" + tm.split("Ngày")[1],"date",ymax])
            
#                                                 break
                                            elif "Thời" in tm :
                                                results[key].append([int(x1),int(y1),"Thời" + tm.split("Thời")[1],"date",ymax])
#                                                 break
                                            else :
                                                results[key].append([int(x1),int(y1),tm,"date",ymax])
#                                                 break
                                    except :
                                        continue
            
            
            
        if len(results["total"]) == 1 :  
            if (hasNumbers(results["total"][0][2])==False or hasCharacters(results["total"][0][2])==False) :
                box1 = results["total"][0]
                ymin_1,ymax_1 = int(box1[1]),int(box1[-1])
                yc_c1 = (ymax_1 + ymin_1)/2
                with open(f"/media/thorpham/PROJECT/OCR-challenge/preprocessing/data_for_submit/private_test/OCR_spelling_correction/{name}","r") as f :
                    file = f.readlines()
                    same_line = []
                    for i in file :
                        bboxs = i.strip().split("\t")
                        label_line = bboxs[8]
                        ymin_2,ymax_2 = int(bboxs[1]),int(bboxs[5])
                        yc_c2 = (ymax_2 + ymin_2)/2
                        if abs(yc_c1-yc_c2) < 10 and (label_line.lower() not in results["total"][0][2].lower()):
#                             results["total"].append([int(bboxs[0]),ymin_2,label_line,ymax_2])
                            same_line.append([int(bboxs[0]),ymin_2,label_line,ymax_2])
                    if len(same_line)==1:
                            results["total"].append(same_line[0])                           
    #sorted(my_list , key=lambda k: [k[1], k[0]])
    company = "|||".join([correct(str(i[2]),field="company") for i in sorted(results["company"] , key=lambda k: k[1])]) 
    adress = "|||".join([correct(str(i[2]),field="address") for i in sorted(results["address"] , key=lambda k: k[1]) if check_address(str(i[2])) is True])
#     print(results["date"])
    date = "|||".join([correct(str(i[2]).split("Số HĐ")[0],field="date") for i in sorted(results["date"] , key=lambda k: k[0])])
#     print("--->",date)
    total = "|||".join([correct(str(i[2]).replace("UND","VND"),field="total") for i in sorted(results["total"] , key=lambda k: k[0])])
    labels_save =  company+"|||"+ adress + "|||" + date + "|||" + total
    print("="*20,name_id,"="*20)
    print(labels_save)
    print("-"*50)
    data_frame.append([name_id,0.5,labels_save])
    date_str.append([name_id,0.5,date])

==================== mcocr_private_145120aahvv.jpg ====================
MINIMART ANAN|||Thu Chợ Sủi Phù Thị Gia Lâm Thị|||Ngày: 13/08/2020-17:52|||Tổng tiền|||146,010
--------------------------------------------------
==================== mcocr_private_145120adyuf.jpg ====================
VỀN VŨ|||Cấm Phà Quảng Ninh|||ĐƯƠNG3/9/39|||Ngày:|||25/07/2020 16:32.52|||Tổng tiền (VND):|||110,000
--------------------------------------------------
==================== mcocr_private_145120aedhd.jpg ====================
|||UK,VINH TU/ M2H|||ĐỊA CHỈ ĐẠI LÝ|||11:15:75|||
--------------------------------------------------
==================== mcocr_private_145120ahyvq.jpg ====================
SIEU THI BACH HOA TONG HOP|||Sò 5 Căm Tây - Cầm Phá - AN|||Ngày: 09/08/2020-16:|||10 Tch 020,3781199.Website|||Tổng cộng (đã gồm VAT):|||52,000
--------------------------------------------------
==================== mcocr_private_145120amcnb.jpg ====================
VinCommerce|||VM+ QNH Tổ 7, khu Minh Tiến A|||

==================== mcocr_private_145120vcrvo.jpg ====================
SIEU THI BACH HOA TONG HOP|||Số 5 Cầm Tây - Cầm Phá - QN|||Ngày: 09/08/2020-15:|||Tổng cộng (đã gồm VAT):|||30,000
--------------------------------------------------
==================== mcocr_private_145120vogtr.jpg ====================
MINIMART ANAN|||Chợ Sủi Phú Thị Gia Lâm||||||Tổng tiền|||293,012
--------------------------------------------------
==================== mcocr_private_145120vvopk.jpg ====================
|||||||||THANH TÙAN|||TAI QUẤT
--------------------------------------------------
==================== mcocr_private_145121cjtsq.jpg ====================
MINIMART ANAN|||Chợ Sủi Phú Thị Gia Lâm|||Ngày: 10/08/2020-16:33|||Tổng tiền|||386,026
--------------------------------------------------
==================== mcocr_private_145121cqrlq.jpg ====================
||||||Ngay: 17/09/2019 18:33:45|||Tong so tien thanh toan:|||144,408.20
--------------------------------------------------
===============

==================== mcocr_private_145120epens.jpg ====================
VinCommerce|||VM+ QNH Tổ 7, Khu Minh Tiến A|||Tổ 7 , Khu Minh Tiến A|||P. Cẩm Bình, TP. Cầm Phả, QNH|||Ngày:13/08/2020|||Thời gian:16:13|||
--------------------------------------------------
==================== mcocr_private_145120etjwj.jpg ====================
||||||NAI|||200.000 Đ
--------------------------------------------------
==================== mcocr_private_145120etnlw.jpg ====================
MINIMART ANAN|||Chợ Sủi Phú Thị Gia Lâm|||Ngày: 16/08/2020-08:44|||Tổng cộng:|||257,000
--------------------------------------------------
==================== mcocr_private_145120euvko.jpg ====================
CỬA HÀNG NĂM OÁNH|||Thôn Phú Thuỵ, Xã Phú Thị, Huyện Gia Lâm, TP Hà Nội|||Ngày: 12-08-2020|||Cộng tiền hàng|||445,000
--------------------------------------------------
==================== mcocr_private_145120ezfnf.jpg ====================
TÊN ĐẠI LÝ:|||VINMART|||và Địa Quí Sĩ|||TỔ 7 NHU MINH TIÊN A,P.CA|||

==================== mcocr_private_145121vwcbq.jpg ====================
Saigon Co.op|||Co.op Food HN THE K-PARK|||"Thang 1,5H42 - Toa K3,Cong trinh chung cua tri|||thanh ket hop dich vu tren o dat H-cT2 thuoc thu|||Du an Dau tu xay dung Khu nha o Hi Brand|||tai Khu do thi moi Van Phu,|||Ngay: 20/07/2017 10:25:34|||Tong so tien thanh toan:|||27,291.20
--------------------------------------------------
==================== mcocr_private_145121vztxn.jpg ====================
VINMART|||ĐỊA CHỈ:|||NGÀY:|||TỔNG:|||100.000 Đ
--------------------------------------------------
==================== mcocr_private_145121wfclu.jpg ====================
VinCommerce|||VM+ QNH Tổ 7, Khu Minh Tiến A|||Tổ 7, Khu Minh Tiến A|||P. Cấm Bình, TP. Cầm Pha, QNH|||Ngày bán:09/08/2020 11:13|||TỔNG TIỀN PHẢI T.TOÁN|||45.000
--------------------------------------------------
==================== mcocr_private_145121wqvca.jpg ====================
TÊN ĐẠI LÝ:|||"THỊ TÚ 8 KHU 3B,P.CÂN TRUNG,1"|||13/08/2020 19:12:43|||

In [472]:
# data_frame.append(["mcocr_val_145114unyae.jpg",0.5,"|||||||||"])
columns = ["img_id","anno_image_quality","anno_texts"]
df = pd.DataFrame(data_frame,columns=columns)
print(df.head())
print(len(df))
df.to_csv("results.csv",index=None,columns=columns)

                          img_id  anno_image_quality  \
0  mcocr_private_145120aahvv.jpg                 0.5   
1  mcocr_private_145120adyuf.jpg                 0.5   
2  mcocr_private_145120aedhd.jpg                 0.5   
3  mcocr_private_145120ahyvq.jpg                 0.5   
4  mcocr_private_145120amcnb.jpg                 0.5   

                                          anno_texts  
0  MINIMART ANAN|||Thu Chợ Sủi Phù Thị Gia Lâm Th...  
1  VỀN VŨ|||Cấm Phà Quảng Ninh|||ĐƯƠNG3/9/39|||Ng...  
2  |||UK,VINH TU/ M2H|||ĐỊA CHỈ ĐẠI LÝ|||11:15:75|||  
3  SIEU THI BACH HOA TONG HOP|||Sò 5 Căm Tây - Cầ...  
4  VinCommerce|||VM+ QNH Tổ 7, khu Minh Tiến A|||...  
390


In [286]:
# original = glob.glob("/media/thorpham/PROJECT/OCR-challenge/data_train/mcocr_public_train_test_shared_data/mcocr_val_data/val_images/*")
# image_orignal = []
# for i in original :
#     name = os.path.basename(i)
#     image_orignal.append(name)

In [21]:
len(original)

464590

In [36]:
data = pd.read_csv("/media/thorpham/PROJECT/OCR-challenge/preprocessing/graph_model/results.csv")
data.head()

,img_id,anno_image_quality,anno_texts
0,mcocr_private_145120aahvv.jpg,0.5,MINIMART ANAN|||Thu Chợ Sủi Phù Thị Gia Lâm Th...
1,mcocr_private_145120adyuf.jpg,0.5,|||Cấm Phà Quảng Ninh|||ĐƯỜNG 3/9/39|||Ngày:||...
2,mcocr_private_145120aedhd.jpg,0.5,"||||||11:15:75|||TỔNG CỘNG:|||UND63, 200"
3,mcocr_private_145120ahyvq.jpg,0.5,sieu thi bach hoa tong hop|||Sò 5 Căm Tây - Cầ...
4,mcocr_private_145120amcnb.jpg,0.5,"Vincommerce|||VM+ QNH Tố 7, Khu Minh Tiền A|||..."


In [8]:
len(original)

391

In [32]:
data

,img_id,anno_image_quality,anno_texts
0,mcocr_val_145114anqqj.jpg,0.5,Joma|||43 Tô Ngọc Vân ( West Lake|||Bakerycafé...
1,mcocr_val_145114aszbc.jpg,0.5,Vincommerce|||VM% QNH Số 463 Tổ 66 Khu Diêm Th...
2,mcocr_val_145114babwd.jpg,0.5,"Vincommerce|||Số 112 Thanh Niên, P. Cầm Thành|..."
3,mcocr_val_145114budzl.jpg,0.5,MINIMART ANAN|||Thợ Sùi Phú Thị Gia Lâm Đó|||N...
4,mcocr_val_145114cijwj.jpg,0.5,MINIMART ANAN|||Chợ Súi Phú Thị Gia Lâm Đó|||N...
...,...,...,...
386,mcocr_val_145115zqplz.jpg,0.5,"VM% HNI 32 Láng Trung|||Số 32 ngô Láng Trung, ..."
387,mcocr_val_145115zuzoa.jpg,0.5,MINIMART ANAN|||Chợ Sủi Phú Thị Gia Lâm|||Ngày...
388,mcocr_val_145115zwqxr.jpg,0.5,"Vincommerce|||Số 112 Thanh Niên, P. Cảm Thành|..."
389,mcocr_val_145115zzaix.jpg,0.5,TÊN ĐAỊ LY|||THỊ TỔ 7 KM MINH TIÊN A.P.CA THỊ|...


In [473]:
df1 = pd.read_csv("/media/thorpham/PROJECT/OCR-challenge/preprocessing/data_for_submit/private_test/mcocr_test_samples_df.csv")
df1.head()

,img_id,anno_image_quality,anno_texts
0,mcocr_private_145120ypcjr.jpg,0.5,abc abc abc
1,mcocr_private_145120euvko.jpg,0.5,abc abc abc
2,mcocr_private_145120aedhd.jpg,0.5,abc abc abc
3,mcocr_private_145120rtovn.jpg,0.5,abc abc abc
4,mcocr_private_145120ywkur.jpg,0.5,abc abc abc


In [474]:
df2 = pd.read_csv("/media/thorpham/PROJECT/OCR-challenge/preprocessing/graph_model/results.csv")
df2.head()

,img_id,anno_image_quality,anno_texts
0,mcocr_private_145120aahvv.jpg,0.5,MINIMART ANAN|||Thu Chợ Sủi Phù Thị Gia Lâm Th...
1,mcocr_private_145120adyuf.jpg,0.5,VỀN VŨ|||Cấm Phà Quảng Ninh|||ĐƯƠNG3/9/39|||Ng...
2,mcocr_private_145120aedhd.jpg,0.5,"|||UK,VINH TU/ M2H|||ĐỊA CHỈ ĐẠI LÝ|||11:15:75|||"
3,mcocr_private_145120ahyvq.jpg,0.5,SIEU THI BACH HOA TONG HOP|||Sò 5 Căm Tây - Cầ...
4,mcocr_private_145120amcnb.jpg,0.5,"VinCommerce|||VM+ QNH Tổ 7, khu Minh Tiến A|||..."


In [475]:
df_final = df1.reset_index()[['index', 'img_id']].merge(df2, on='img_id', how='left').fillna('|||||||||').sort_values(by='index').drop(columns='index')

In [476]:
df_final.to_csv("submit_final/results.csv",index=None)

In [113]:
spelling_correction = pd.read_csv("/media/thorpham/PROJECT/OCR-challenge/preprocessing/spelling_correction.csv")
spelling_correction.head()

,Unnamed: 0,predict,grouth_True,class_id
0,0,NHÀ SÁCH GD-TC CẨM PHẢ,NHÀ SÁCH GD-TC CẨM PHẢ,SELLER
1,1,Đc: 212 Đường Trấn Phú-Cẩm Phả Nhau,ĐC: 212 Đường Trần Phú - Cẩm Phả,ADDRESS
2,2,Thời gian:16:16:02 - 14/08/2020,Thời gian: 15:16:02 - 14/08/2020,TIMESTAMP
3,3,Tổng số thanh toán:,Tổng số thanh toán,TOTAL_COST
4,4,246.000,246.000đ,TOTAL_COST


In [123]:
spelling_correction_address = {}
spelling_correction_company = {}
for i in range(len(spelling_correction)) :
    class_id = spelling_correction["class_id"].iloc[i]
    if class_id == "ADDRESS":
        if spelling_correction["predict"].iloc[i] not in spelling_correction_address : 
            spelling_correction_address[spelling_correction["predict"].iloc[i]] = spelling_correction["grouth_True"].iloc[i]
    if class_id == "SELLER":
        if spelling_correction["predict"].iloc[i] not in spelling_correction_company : 
            spelling_correction_company[spelling_correction["predict"].iloc[i]] = spelling_correction["grouth_True"].iloc[i]

In [316]:
paths = glob.glob("rs1/*")
count = 0 
for path in paths :
    results = {"company":[],"address":[],"date":[],"total":[]}
    outputs   = {"company":"","address":"","date":"","total":""}
    name = os.path.basename(path)
    name_id = name.replace("txt","jpg")
#     image = cv2.imread(f"/media/thorpham/PROJECT/OCR-challenge/preprocessing/data_for_submit/data_test/{name_id}")
#     image_show = image.copy()
   
    with open(path,"r") as file :
        data = file.readlines()
        with open(f"result_spelling_correction/{name}","w")  as f :
            for line in data:
                tmp = line.strip().split("\t")
                text = tmp[10]
                class_id = tmp[11]
                if class_id == "company":
                    check1 = company_dictionary.get(text)
                    if check1 is not None :
                        tmp[10] = company_dictionary.get(text)
                if class_id == "address":
                    check2 = address_dictionary.get(text)
                    if check2 is not None :
                        tmp[10] = address_dictionary.get(text)
                f.write("\t".join([str(i) for i in tmp]) + "\n")
                    

In [115]:
count

0

In [215]:
paths = glob.glob("rs1/*")
count = 0 
for path in paths :
    results = {"company":[],"address":[],"date":[],"total":[]}
    outputs   = {"company":"","address":"","date":"","total":""}
    name = os.path.basename(path)
    name_id = name.replace("txt","jpg")
#     image = cv2.imread(f"/media/thorpham/PROJECT/OCR-challenge/preprocessing/data_for_submit/data_test/{name_id}")
#     image_show = image.copy()
    with open(path,"r") as file :
        data = file.readlines()
        for line in data:
            tmp = line.strip().split("\t")
            text = tmp[10]
            class_id = tmp[11]
            if class_id == "address":
                x = address_dictionary.get(text)
                if x is not None :
                    count +=1
                    print("="*20,path,"="*20)
                    print("="*10,text,"="*10)
                    print("="*10,address_dictionary.get(text),"="*10)
#                     tmp[10] = address_dictionary.get(text)
#             print(tmp)

                    

==================== rs1/mcocr_private_145120amcnb.txt ====================
========== VM% QNH Tố 7, Khu Minh Tiến A ==========
========== VM+ QNH Tổ 7, khu Minh Tiến A ==========
==================== rs1/mcocr_private_145120apnkj.txt ====================
========== Chợ Sùi Phú Thị Gia Lâm ==========
========== Chợ Sủi Phú Thị Gia Lâm ==========
==================== rs1/mcocr_private_145120bdbex.txt ====================
========== Chợ Sủi Phú Thị Gia Lâm ==========
========== Chợ Sủi Phú Thị Gia Lâm ==========
==================== rs1/mcocr_private_145120bgmgl.txt ====================
========== VM4 QNH Tổ 7, Khu Minh Tiến A ==========
========== VM+ QNH Tổ 7, Khu Minh Tiến A ==========
==================== rs1/mcocr_private_145120biprv.txt ====================
========== VM% QNH 1060-1062 Trần Phú ==========
========== VM+QNH 1060-1062 Trần Phú ==========
==================== rs1/mcocr_private_145120bsynb.txt ====================
========== P. Cẩm Bình, TP. Cẩm Phà, QNH ==========
===

==================== rs1/mcocr_private_145120qsskw.txt ====================
========== 28 Phan Chu Trinh, Hiệp Phú, Quận 9, Hồ ==========
========== 28 Phan Chu Trinh, Hiệp Phú, Quận 9, Hồ ==========
==================== rs1/mcocr_private_145120qxnqk.txt ====================
========== VM% QNH Tô 7, Khu Minh Tiến A ==========
========== VM+QNH Tổ 7, Khu Minh Tiến A ==========
==================== rs1/mcocr_private_145120qxnqk.txt ====================
========== Tổ 7, Khu Minh Tiến A ==========
========== Tổ 7, Khu Minh Tiến A ==========
==================== rs1/mcocr_private_145120rfgjj.txt ====================
========== VM% QNH 112 Thanh Niên ==========
========== VM+ QNH 112 Thanh Niên ==========
==================== rs1/mcocr_private_145120rfgjj.txt ====================
========== TP. Càm Phà, Quảng Ninh ==========
========== TP. Cẩm Phả, Quảng Ninh ==========
==================== rs1/mcocr_private_145120rgoiu.txt ====================
========== 28 Phan Chu Trinh, Hiệp Phú, Quận 9,

In [190]:
count

306

In [149]:
spelling_correction.head()

,Unnamed: 0,predict,grouth_True,class_id
0,0,NHÀ SÁCH GD-TC CẨM PHẢ,NHÀ SÁCH GD-TC CẨM PHẢ,SELLER
1,1,Đc: 212 Đường Trấn Phú-Cẩm Phả Nhau,ĐC: 212 Đường Trần Phú - Cẩm Phả,ADDRESS
2,2,Thời gian:16:16:02 - 14/08/2020,Thời gian: 15:16:02 - 14/08/2020,TIMESTAMP
3,3,Tổng số thanh toán:,Tổng số thanh toán,TOTAL_COST
4,4,246.000,246.000đ,TOTAL_COST


In [155]:
from collections import Counter

In [172]:
spelling_correction_company = {}
for i in range(len(spelling_correction)) :
    class_id = spelling_correction["class_id"].iloc[i]
    if class_id == "SELLER":
        idx = spelling_correction["predict"].iloc[i]
        value = spelling_correction["grouth_True"].iloc[i]
        if idx not in spelling_correction_company:
            spelling_correction_company[idx] = Counter([value])
        else :
            spelling_correction_company[idx].update([value])

In [186]:
spelling_correction_address = {}
for i in range(len(spelling_correction)) :
    class_id = spelling_correction["class_id"].iloc[i]
    if class_id == "ADDRESS":
        idx = spelling_correction["predict"].iloc[i]
        value = spelling_correction["grouth_True"].iloc[i]
        if idx not in spelling_correction_address:
            spelling_correction_address[idx] = Counter([value])
        else :
            spelling_correction_address[idx].update([value])

In [183]:
company_dictionary = {}
for key,val in spelling_correction_company.items():
    company_dictionary[key] = val.most_common()[0][0]

In [187]:
address_dictionary = {}
for key,val in spelling_correction_address.items():
    address_dictionary[key] = val.most_common()[0][0]

In [188]:
address_dictionary

{'Đc: 212 Đường Trấn Phú-Cẩm Phả Nhau': 'ĐC: 212 Đường Trần Phú - Cẩm Phả',
 'Chợ Sủi Phú Thị Gia Làm': 'Chợ Sủi Phú Thị Gia Lâm',
 'Vincom Phạm Ngọc Thạch': 'Vincom Phạm Ngọc Thạch',
 '2, Đượng Phạm Ngọc Thạch': '2, Đường Phạm Ngọc Thạch',
 'Phường Trung Trong Quân Đồng Đa': 'Phường Trung Tự, Quận Đống Đa',
 'Thành phố hà nội': 'Thành phố Hà Nội',
 'VM% QNH Dư án quỹ đất đường sắt': 'VM + QNH Dự án quỹ đất đường sắt',
 'DA QĐ dọc tuyên ĐS Tổ 3 Khu 2': 'DA QĐ doc tuyến ĐS Tổ 3 Khu 2',
 'P. Cầm Sơn, TP. Cảm Phà, T. QN': 'P. Cẩm Sơn, TP. Cẩm Phả, T.QN',
 'VM% QNH Tố 7, Khu Minh Tiến A': 'VM+ QNH Tổ 7, khu Minh Tiến A',
 'Tổ 7, Khu Minh Tiến Ả': 'Tổ 7, Khu Minh Tiến A',
 'P. Cầm Bình TP. Cảm Pha, QNH': 'P. Cẩm Bình, TP. Cẩm Phả, QNH',
 'Chợ Sủi Phú Thị Gia Lâm': 'Chợ Sủi Phú Thị Gia Lâm',
 'VM% GNH 690 Trần Phú': 'VM + QNH 590 Trần Phú',
 'Sỏ 590 Trần Phú, P. Cẩm Thủy': 'Số 590 Trần Phú.P.Cẩm Thuy',
 'TP. Cẩm Phà, Quảng Ninh': 'TP.Cẩm Phả, Quảng Ninh',
 'Chợ Sùi Phú Thị Gia Lâm': 'Chợ Sủi

In [260]:
import json
import re

In [259]:
def hasCharacters(s):
    s = re.sub(r'[^\w\s]','',s)
    return all(char.isalpha() for char in s)

In [266]:
spelling_correction_total = {}
spelling_correction_date = {}
for i in range(len(spelling_correction)) :
    class_id = spelling_correction["class_id"].iloc[i]
    if class_id == "TOTAL_COST":
        if spelling_correction["predict"].iloc[i] not in spelling_correction_total : 
            if 1 : #hasCharacters(spelling_correction["predict"].iloc[i]):
                spelling_correction_total[spelling_correction["predict"].iloc[i]] = spelling_correction["grouth_True"].iloc[i]
    if class_id == "TIMESTAMP":
        if 1 : #spelling_correction["predict"].iloc[i] not in spelling_correction_date : 
            if hasCharacters(spelling_correction["predict"].iloc[i]):
                spelling_correction_date[spelling_correction["predict"].iloc[i]] = spelling_correction["grouth_True"].iloc[i]

In [311]:
company_patterns = {
"104 TRẦN PHÚ - CẨM": "104 TRẦN PHÚ - CẨM PHẢ",
"104 TRẦN PHÚ - CẨM THẾ": "104 TRẦN PHÚ - CẨM PHẢ",
"BIG T": "BIG C",
"co ciy cp di pt on tip": "CTY CP DT PT CN TTP",
"Co op": "Co.op",
"Co. op": "Co.op",
"Co.0p": "Co.op",
"Cowop": "Co.op",
"It Co.op": "Co.op",
"Colop Food": "Co.op Food",
"Cowop Food": "Co.op Food",
"Co.opHart": "Co.opMart",
"Cong Ty UP": "Cong Ty CP",
"CÁM PHẢ": "CẨM PHẢ",
"CÂM PHẢ": "CẨM PHẢ",
"CÂM PHẢ THỊ": "CẨM PHẢ",
"CẤM PHẢ": "CẨM PHẢ",
"CẤM THẾ": "CẨM PHẢ",
"CẨM PHÀ": "CẨM PHẢ",
"CẨM PHÁP": "CẨM PHẢ",
"CẨM PHẢ PHẢ": "CẨM PHẢ",
"CẨM PHẢI": "CẨM PHẢ",
"CẨM PHẢNG": "CẨM PHẢ",
"GD.TC": "GD-TC",
"HA NOII": "HA NOI",
"JahaTafe": "JahaCafe",
"LD TNH KFC": "LD TNHH KFC",
"A MINIMART": "MINIMART",
"A MINIMART ANAN": "MINIMART ANAN",
"A MINIMART ANANT": "MINIMART ANAN",
"IN MINIMART ANAN 1": "MINIMART ANAN",
"IN MINIMART ANANIA": "MINIMART ANAN",
"MINIMART ANAN 1": "MINIMART ANAN",
"MINIMART ANANA": "MINIMART ANAN",
"MINIMART ANANAN": "MINIMART ANAN",
"MINIMART ANANIA": "MINIMART ANAN",
"MINIMARTANAN": "MINIMART ANAN",
"MINIMARTANANA": "MINIMART ANAN",
"MINIMARTANANT": "MINIMART ANAN",
"Vinilart": "Minimart",
"VÀ NHÀ SÁCH": "NHÀ SÁCH",
"ĐÃ NHÀ SÁCH": "NHÀ SÁCH",
"VÀ NHÀ SÁCH GD-TC CÂM PHẢ THỊ": "NHÀ SÁCH GD-TC CẨM PHẢ",
"VÀ NHÀ SÁCH GD-TC CẨM PHÀ": "NHÀ SÁCH GD-TC CẨM PHẢ",
"VÀ NHÀ SÁCH GD-TC CẨM PHẢI": "NHÀ SÁCH GD-TC CẨM PHẢ",
"ĐÃ NHÀ SÁCH GD-TC CÂM PHẢ": "NHÀ SÁCH GD-TC CẨM PHẢ",
"ĐÃ NHÀ SÁCH GD-TC CẨM PHÁP": "NHÀ SÁCH GD-TC CẨM PHẢ",
"ĐÃ NHÀ SÁCH GD-TC CẨM PHẢ": "NHÀ SÁCH GD-TC CẨM PHẢ",
"ĐÃ NHÀ SÁCH GD-TC CẨM PHẢI": "NHÀ SÁCH GD-TC CẨM PHẢ",
"ĐÃ NHÀ SÁCH GD-TC CẨM PHẢNG": "NHÀ SÁCH GD-TC CẨM PHẢ",
"PHÀ": "PHẢ",
"PHÁ": "PHẢ",
"PHÓ MỎ": "PHỐ MỎ",
"PHÔ MỎ": "PHỐ MỎ",
"PHỔ MÔI": "PHỐ MỎ",
"PHỔ MỎ": "PHỐ MỎ",
"TRONG SCTC": "SCTC",
"Saigon Co.Op": "Saigon Co.op",
"TNH KFC": "TNHH KFC",
"Tr Thời trang xuất khẩu M2": "TT Thời trang",
"In the coffee house": "The coffee house",
"wor the coffee house": "The coffee house",
"Thành phố Cấm": "Thành phố Cẩm",
"TÍN Đ": "TÊN Đ",
"TÊN DAI LY": "TÊN ĐẠI LÝ",
"TÊN MÁI LY": "TÊN ĐẠI LÝ",
"TÊN MỊ IV": "TÊN ĐẠI LÝ",
"TÊN MỊ LÝ": "TÊN ĐẠI LÝ",
"TÊN DAI LY:": "TÊN ĐẠI LÝ:",
"TÊN ĐÀI LÝ:": "TÊN ĐẠI LÝ:",
"TÍN ĐÀI LÝ:": "TÊN ĐẠI LÝ:",
"TỔ 7 NHU MINH TIÊN A": "TỔ 7 KHU MINH TIẾN A",
"UNCOM": "VINCOM",
"LUINMART": "VINMART",
"VI MART": "VINMART",
"VINNANT": "VINMART",
"VINNART": "VINMART",
"VINTART": "VINMART",
"WINMART": "VINMART",
"VIA%": "VM+",
"VIM%": "VM+",
"Vùi": "VM+",
"Uncommerce": "Vincommerce",
"Uncommerced": "Vincommerce",
"VIncommerce": "Vincommerce",
"Vincommering": "Vincommerce",
"Vncommerce": "Vincommerce",
"Yincommerce": "Vincommerce",
"XA LO HA No": "XA LO HA NOI",
"dec offee to gon": "coffe to go",
"flaza": "plaza",
"sau sieu thi": "sieu thi",
"In the coffee": "the coffee",
"ĐA] LY": "ĐẠI LÝ",
"ĐAỊ LY": "ĐẠI LÝ",
"ĐHỊ LÝ": "ĐẠI LÝ",
"ĐÀI LÝ": "ĐẠI LÝ",
"ĐƯU ĐÀM CHAY": "ƯU ĐÀM CHAY",
}

In [312]:
for k,v in company_patterns.items() :
    if k not in company_dictionary.items():
        company_dictionary[k] =  v

In [315]:
for k,v in address_pattern.items() :
    if k not in address_dictionary.items():
        company_dictionary[k] =  v

In [367]:
results["date"]

[[15, 245, 'Ngày bán:15/08/2020 06:10', 'date', '269'],
 [15, 245, 'Ngày bán:15/08/2020 06:10', 'date', '418']]

In [370]:
x =  [i[:3] for i in results["date"]]

In [371]:
x

[[15, 245, 'Ngày bán:15/08/2020 06:10'],
 [15, 245, 'Ngày bán:15/08/2020 06:10']]

In [416]:
correct("ĐỊA ONỈ ĐẠI LÝ",field="company")

'ĐỊA CHỈ ĐẠI LÝ'

In [438]:
import pandas as pd
import numpy as np


def calc_line_count(res_df):
    if res_df.shape[0] < 1:
        return pd.Series([])
    df = res_df.copy()
    df['dc'] = df['c_on_vh'] - df['c_on_vh'].shift()
    h_min = df.eval("bot_on_vh - top_on_vh").abs().min()
    distances = df['dc'][df['dc'] >= h_min]
    if 'address' in df['label_copy']:
        scaler = 0.8
    else:
        scaler = 1
    if distances.shape[0] > 0:
        y_thresh = distances.min()
        df['new_line'] = (df['dc'].isna()) | (df['dc'] >= y_thresh*scaler)
        df['line_count'] = df['new_line'].cumsum()
    else:
        df['line_count'] = 1
    return df[['line_count']]


def numpy_2_line_number(np_arr):
    columns = ['x_tl', 'y_tl', 'x_tr', 'y_tr', 'x_br', 'y_br', 'x_bl', 'y_bl', 'label']
    df = pd.DataFrame(np_arr, columns=columns)
    df['xc'] = df[['x_tl', 'x_tr', 'x_br', 'x_bl']].mean(axis=1)
    df['yc'] = df[['y_tl', 'y_tr', 'y_br', 'y_bl']].mean(axis=1)

    df['x_top'] = df[['x_tl', 'x_tr']].mean(axis=1)
    df['y_top'] = df[['y_tl', 'y_tr']].mean(axis=1)
    df['x_bot'] = df[['x_bl', 'x_br']].mean(axis=1)
    df['y_bot'] = df[['y_bl', 'y_br']].mean(axis=1)

    xh = df.eval("x_bot - x_top").median()
    yh = df.eval("y_bot - y_top").median()
    vh = np.sqrt(xh ** 2 + yh ** 2)
    # projection
    df['top_on_vh'] = df.eval("(x_top * @xh + y_top * @yh) / @vh")
    df['bot_on_vh'] = df.eval("(x_bot * @xh + y_bot * @yh) / @vh")
    df['c_on_vh'] = df.eval("(xc * @xh + yc * @yh) / @vh")
    # if (df['c_on_vh'] < 0).sum() > 0:
    #     print("check", ocr_res_file)
    df['c_on_vh'] = df['c_on_vh'].abs()
    df.sort_values(by='c_on_vh', inplace=True)
    df['label_copy'] = df['label']
    if df.label.nunique() > 1:
        if df.label.nunique() < df.shape[0]:
            line_count = df.groupby('label').apply(calc_line_count) \
                .rename_axis(['label', 'index']) \
                .reset_index() \
                .drop(columns='label') \
                .set_index('index')
            df = df.merge(line_count, left_index=True, right_index=True)
        else:
            df['line_count'] = 1
    else:
        df['line_count'] = calc_line_count(df)
    sorted_df = df.sort_values(by=['label', 'line_count', 'c_on_vh'])
    return sorted_df[columns].values

In [463]:
# data_frame.append(["mcocr_val_145114unyae.jpg",0.5,"|||||||||"])
columns = ["img_id","score","anno_texts"]
df = pd.DataFrame(date_str,columns=columns)
df.to_csv("results.csv",index=None,columns=columns)